In [ ]:
# ------------------------------ IMPORTS ----------------------------------------- #

# Built-in 
import os
import glob
import numpy as np
import matplotlib.pyplot as plt

# Own Libs
from utils import read_Tumag
from field_stop_finder import find_fieldstop, fieldstopping_and_shifting
from master_dark import compute_master_darks
from master_flatfield import compute_master_flat_field
import image_handler as ih
from demodulation import demodulate

In [ ]:
# Testeo
darks_c1 = sorted(glob.glob(f"/home/pablo/Desktop/TuMAGDATA/Darks/*_0_*")) 
darks_c2 = sorted(glob.glob(f"/home/pablo/Desktop/TuMAGDATA/Darks/*_1_*")) 

ground_flat_path_c1 = "/home/pablo/Desktop/SuObsTEsts/DATA/27/2024_05_01_10_10_36_149_0_690.img"
ground_flat_path_c2 = "/home/pablo/Desktop/SuObsTEsts/DATA/27/2024_05_01_10_10_36_149_1_690.img"

flats_c1 = sorted(glob.glob(f"/home/pablo/Desktop/SuObsTEsts/DATA/29/*_0_*")) 
flats_c2 = sorted(glob.glob(f"/home/pablo/Desktop/SuObsTEsts/DATA/29/*_1_*")) 

In [ ]:
Obs_folder = "/home/pablo/Desktop/TuMAGDATA/FS_modes_2_02/"

nobs = 2
nmods = 4
nlambda = 8

all_images = sorted(glob.glob(f"{Obs_folder}*"))
mode_222_ims = []
for im in all_images:
    I, H = read_Tumag(im)
    oc = H["Observation_Counter"]
    if oc == 222:
        mode_222_ims.append(im)

mode_222_ims = np.array(mode_222_ims)
reshaped_paths = np.array(mode_222_ims).reshape(nlambda, nmods, 2 )
ob = ih.nominal_observation("2.02", mode_222_ims)
data = ob.get_data()

In [ ]:
fs_obs_c1, fs_obs_c2 = find_fieldstop(mode = "auto", cam1 = data[0, -1, 0], cam2 = data[0, -1, 1], plot_flag = False, verbose = True)

ground_flat_cam1, _ = read_Tumag(ground_flat_path_c1)
ground_flat_cam2, _ = read_Tumag(ground_flat_path_c2)
fs_ground_c1, fs_ground_c2 = find_fieldstop(mode = "auto", cam1 = ground_flat_cam1, cam2 = ground_flat_cam2, plot_flag = False, verbose = True)

flats_c2 = sorted(glob.glob(f"/home/pablo/Desktop/SuObsTEsts/DATA/27/*_1_*")) 
flats_c1 = sorted(glob.glob(f"/home/pablo/Desktop/SuObsTEsts/DATA/27/*_0_*")) 

dc_obs = compute_master_darks(darks_c1, darks_c2, fs_obs_c1, fs_obs_c2, verbose = True)
ff = compute_master_flat_field(flats_c1, flats_c2, np.zeros((2, 2016, 2016)), fs_ground_c1, fs_ground_c2, verbose = True)

#fs_ground_c1[0][1] -= 1


#fs_obs_c2 = np.array(fs_obs_c2)
#fs_obs_c2[1] += 20

In [ ]:
"""corrected = np.zeros((2, nmods, nlambda, 1666, 1664))

for mod in range(nmods):
    for lamb in range(nlambda):
        corrected[0, mod, lamb] = (data[0, lamb, mod, fs_obs_c1[0][0]:fs_obs_c1[0][1], fs_obs_c1[1][0]:fs_obs_c1[1][1]] - dc_obs[0, fs_obs_c1[0][0]:fs_obs_c1[0][1], fs_obs_c1[1][0]:fs_obs_c1[1][1]]) / ff[0,fs_ground_c1[0][0]:fs_ground_c1[0][1], fs_ground_c1[1][0]:fs_ground_c1[1][1]]
        corrected[1, mod, lamb] = (data[1, lamb, mod, fs_obs_c2[0][0]:fs_obs_c2[0][1], fs_obs_c2[1][0]:fs_obs_c2[1][1]] - dc_obs[1, fs_obs_c2[0][0]:fs_obs_c2[0][1], fs_obs_c2[1][0]:fs_obs_c2[1][1]]) / ff[1,fs_ground_c2[0][0]:fs_ground_c2[0][1], fs_ground_c2[1][0]:fs_ground_c2[1][1]]
fig, axs = plt.subplots(2, 3, figsize = (10, 5))

axs[0, 0].imshow(dc_obs[0, fs_obs_c1[0][0]:fs_obs_c1[0][1], fs_obs_c1[1][0]:fs_obs_c1[1][1]], vmin = 2000, cmap = "gray")
axs[1, 0].imshow(dc_obs[1, fs_obs_c2[0][0]:fs_obs_c2[0][1], fs_obs_c2[1][0]:fs_obs_c2[1][1]], vmin = 2000, cmap = "gray")

axs[0, 1].imshow(ff[0,fs_ground_c1[0][0]:fs_ground_c1[0][1], fs_ground_c1[1][0]:fs_ground_c1[1][1]], cmap = 'inferno')
axs[1, 1].imshow(ff[1,fs_ground_c2[0][0]:fs_ground_c2[0][1], fs_ground_c2[1][0]:fs_ground_c2[1][1]], cmap = 'inferno')

axs[0, 2].imshow(corrected[0, 0, -1], cmap = "gray")
axs[1, 2].imshow(corrected[0, 0, -1], cmap = "gray")

plt.tight_layout()"""

corrected = np.zeros((2, nmods, nlambda, 2016,2016))
for mod in range(nmods):
    for lamb in range(nlambda):

        obs_c1, obs_c2 = fieldstopping_and_shifting(data[0, lamb, mod], data[1, lamb, mod], fs_obs_c1, fs_obs_c2)
        dc1, dc2 = fieldstopping_and_shifting(dc_obs[0], dc_obs[1], fs_obs_c1, fs_obs_c2)

        corrected[1, mod, lamb] = (obs_c1 - dc1)
        corrected[0, mod, lamb] = (obs_c2 - dc2)


#fig, axs = plt.subplots(2, 3, figsize = (10, 5))

del data, ff, dc_obs

In [ ]:
#demod, dual = demodulate(corrected[:, :, 3], 1666, 1664, 4, 8, filt = "525.02", mode = 'single_wavelength') 
demod, dual = demodulate(corrected[:, :, 3],2016,2016, 4, 8, filt = "525.02", mode = 'single_wavelength') 

In [ ]:
fig, axs = plt.subplots(2, 2, figsize = (15, 15))

axs[0, 0].imshow(demod[0, 0], cmap = "gray")
axs[0, 1].imshow(demod[0, 1], cmap = "gray")
axs[1, 0].imshow(demod[0, 2], cmap = "gray")
axs[1, 1].imshow(demod[0, 3], cmap = "gray")
plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize = (15, 15))

axs[0, 0].imshow(dual[0], cmap = "gray")
axs[0, 1].imshow(dual[1], cmap = "gray")
axs[1, 0].imshow(dual[2], cmap = "gray")
axs[1, 1].imshow(dual[3], cmap = "gray")


plt.tight_layout()


In [ ]:
np.shape(data[0, :, 0] )
